In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/home/clay/userdata/title_generation


In [2]:
from pathlib import Path
from pyabc import pyabc
import shutil
import pandas as pd
import numpy as np
import json

from data_utils import prepare_abc

import tiktoken
from openai.embeddings_utils import get_embedding
from sentence_transformers import SentenceTransformer

In [9]:
df_embedding_all = pd.read_csv('unique_titles_with_embedding.csv')

In [10]:
len(list(df_embedding_all["Title"]))

5417

In [35]:
df_embedding_all.head()

,Title,combined,combined_genre,ST_embedding_withsongtitle,OA_embedding_withsongtitle,OA_embedding_withsongtitle_genre,OA_embedding_titleonly,Language_azure
0,The Devil In The Kitchen,Song title: The Devil In The Kitchen,Song title: The Devil In The Kitchen; Genre: i...,"[0.030707573518157005, 0.01104187685996294, -0...","[0.007431528531014919, -0.017284130677580833, ...","[-0.00202469015493989, -0.010335808619856834, ...","[0.006531674414873123, -0.01711384579539299, -...",NaN
1,10th Bat Crossing Rhine,Song title: 10th Bat Crossing Rhine,Song title: 10th Bat Crossing Rhine; Genre: ir...,"[-0.022036369889974594, -0.07493089139461517, ...","[0.0002703349164221436, -0.027832742780447006,...","[-0.006309073884040117, -0.026604685932397842,...","[0.002080520149320364, -0.020544277504086494, ...",NaN
2,1st August,Song title: 1st August,Song title: 1st August; Genre: irish folk music,"[-0.005911257117986679, 0.06030600517988205, -...","[-0.016317373141646385, -0.00714963860809803, ...","[-0.020458517596125603, -0.0008305144729092717...","[-0.013012313283979893, 0.0048871287144720554,...",NaN
3,2D Or Not 2D,Song title: 2D Or Not 2D,Song title: 2D Or Not 2D; Genre: irish folk music,"[0.044478822499513626, 0.054849039763212204, -...","[-0.015798235312104225, 0.00037473710835911334...","[-0.0210439320653677, 0.0053880284540355206, 0...","[-0.009395561181008816, 0.003268165746703744, ...",NaN
4,The 30 Year,Song title: The 30 Year,Song title: The 30 Year; Genre: irish folk music,"[0.022224457934498787, 0.03378339484333992, -0...","[-0.01499841921031475, -0.022419240325689316, ...","[-0.018276482820510864, -0.016491519287228584,...","[-0.0221587922424078, -0.015429182909429073, -...",NaN


In [38]:
# make empty column for azure language
df_embedding_all['Language_azure'] = np.nan
# put columns in order
df_embedding_all = df_embedding_all.reindex(columns=['Title', 'Language_azure', 'combined', 'combined_genre', 'ST_embedding_withsongtitle',  'OA_embedding_titleonly', 'OA_embedding_withsongtitle', 'OA_embedding_withsongtitle_genre'])

In [39]:
df_embedding_all.head()

,Title,Language_azure,combined,combined_genre,ST_embedding_withsongtitle,OA_embedding_titleonly,OA_embedding_withsongtitle,OA_embedding_withsongtitle_genre
0,The Devil In The Kitchen,NaN,Song title: The Devil In The Kitchen,Song title: The Devil In The Kitchen; Genre: i...,"[0.030707573518157005, 0.01104187685996294, -0...","[0.006531674414873123, -0.01711384579539299, -...","[0.007431528531014919, -0.017284130677580833, ...","[-0.00202469015493989, -0.010335808619856834, ..."
1,10th Bat Crossing Rhine,NaN,Song title: 10th Bat Crossing Rhine,Song title: 10th Bat Crossing Rhine; Genre: ir...,"[-0.022036369889974594, -0.07493089139461517, ...","[0.002080520149320364, -0.020544277504086494, ...","[0.0002703349164221436, -0.027832742780447006,...","[-0.006309073884040117, -0.026604685932397842,..."
2,1st August,NaN,Song title: 1st August,Song title: 1st August; Genre: irish folk music,"[-0.005911257117986679, 0.06030600517988205, -...","[-0.013012313283979893, 0.0048871287144720554,...","[-0.016317373141646385, -0.00714963860809803, ...","[-0.020458517596125603, -0.0008305144729092717..."
3,2D Or Not 2D,NaN,Song title: 2D Or Not 2D,Song title: 2D Or Not 2D; Genre: irish folk music,"[0.044478822499513626, 0.054849039763212204, -...","[-0.009395561181008816, 0.003268165746703744, ...","[-0.015798235312104225, 0.00037473710835911334...","[-0.0210439320653677, 0.0053880284540355206, 0..."
4,The 30 Year,NaN,Song title: The 30 Year,Song title: The 30 Year; Genre: irish folk music,"[0.022224457934498787, 0.03378339484333992, -0...","[-0.0221587922424078, -0.015429182909429073, -...","[-0.01499841921031475, -0.022419240325689316, ...","[-0.018276482820510864, -0.016491519287228584,..."


## Azure

In [8]:
import sys
sys.path.append('Text-Analytics')
from credential import client

"""
Language detection
"""
text_string = [
    "A Deanadh Im",
    "A Mhisg A Chur An Lolig Oirn",
    "A Nation Once Again",
    "A Trip To The Cottage"
]

client = client()
response = client.detect_language(documents=text_string)
print(response)

[DetectLanguageResult(id=0, primary_language=DetectedLanguage(name=Irish, iso6391_name=ga, confidence_score=0.94), warnings=[], statistics=None, is_error=False), DetectLanguageResult(id=1, primary_language=DetectedLanguage(name=Irish, iso6391_name=ga, confidence_score=0.93), warnings=[], statistics=None, is_error=False), DetectLanguageResult(id=2, primary_language=DetectedLanguage(name=English, iso6391_name=en, confidence_score=0.85), warnings=[], statistics=None, is_error=False), DetectLanguageResult(id=3, primary_language=DetectedLanguage(name=English, iso6391_name=en, confidence_score=0.91), warnings=[], statistics=None, is_error=False)]


In [18]:
response[0].primary_language.name

'Irish'

In [41]:
# get client response of titles in every 1000 batches
# put language into df_embedding_all['Language']
from credential import client
client = client()

for i in range(0, len(list(df_embedding_all["Title"])), 1000):
    if i == 5000:
        response = client.detect_language(documents=list(df_embedding_all["Title"][i:]))
    else:
        response = client.detect_language(documents=list(df_embedding_all["Title"][i:i+1000]))
    
    for j in range(len(response)):
        df_embedding_all['Language_azure'][i+j] = response[j].primary_language.name

/tmp/ipykernel_1656674/2622876565.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_embedding_all['Language_azure'][i+j] = response[j].primary_language.name
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [42]:
df_embedding_all.head()

,Title,Language_azure,combined,combined_genre,ST_embedding_withsongtitle,OA_embedding_titleonly,OA_embedding_withsongtitle,OA_embedding_withsongtitle_genre
0,The Devil In The Kitchen,English,Song title: The Devil In The Kitchen,Song title: The Devil In The Kitchen; Genre: i...,"[0.030707573518157005, 0.01104187685996294, -0...","[0.006531674414873123, -0.01711384579539299, -...","[0.007431528531014919, -0.017284130677580833, ...","[-0.00202469015493989, -0.010335808619856834, ..."
1,10th Bat Crossing Rhine,English,Song title: 10th Bat Crossing Rhine,Song title: 10th Bat Crossing Rhine; Genre: ir...,"[-0.022036369889974594, -0.07493089139461517, ...","[0.002080520149320364, -0.020544277504086494, ...","[0.0002703349164221436, -0.027832742780447006,...","[-0.006309073884040117, -0.026604685932397842,..."
2,1st August,English,Song title: 1st August,Song title: 1st August; Genre: irish folk music,"[-0.005911257117986679, 0.06030600517988205, -...","[-0.013012313283979893, 0.0048871287144720554,...","[-0.016317373141646385, -0.00714963860809803, ...","[-0.020458517596125603, -0.0008305144729092717..."
3,2D Or Not 2D,English,Song title: 2D Or Not 2D,Song title: 2D Or Not 2D; Genre: irish folk music,"[0.044478822499513626, 0.054849039763212204, -...","[-0.009395561181008816, 0.003268165746703744, ...","[-0.015798235312104225, 0.00037473710835911334...","[-0.0210439320653677, 0.0053880284540355206, 0..."
4,The 30 Year,English,Song title: The 30 Year,Song title: The 30 Year; Genre: irish folk music,"[0.022224457934498787, 0.03378339484333992, -0...","[-0.0221587922424078, -0.015429182909429073, -...","[-0.01499841921031475, -0.022419240325689316, ...","[-0.018276482820510864, -0.016491519287228584,..."


In [47]:
# save df_embedding_all
df_embedding_all.to_csv('unique_titles_with_embedding_langdetct.csv', index=False)

In [50]:
df_embedding_all = pd.read_csv('unique_titles_with_embedding_langdetct.csv')
df_embedding_all["ST_embedding_withsongtitle"] = df_embedding_all["ST_embedding_withsongtitle"].apply(lambda x: np.array(eval(x)))
df_embedding_all['OA_embedding_withsongtitle'] = df_embedding_all['OA_embedding_withsongtitle'].apply(lambda x: np.array(eval(x)))
df_embedding_all['OA_embedding_withsongtitle_genre'] = df_embedding_all['OA_embedding_withsongtitle_genre'].apply(lambda x: np.array(eval(x)))
df_embedding_all['OA_embedding_titleonly'] = df_embedding_all['OA_embedding_titleonly'].apply(lambda x: np.array(eval(x)))

In [54]:
df_embedding_all['ST_embedding_withsongtitle'][3][18]

0.010504906065762043

In [48]:
# make new dataframe only includes Title and Language_azure
df_language_check = df_embedding_all[['Title', 'Language_azure']]
df_language_check.to_csv('unique_titles_with_language_detect.csv', index=False)

In [59]:
# chek what values have in Language_azure columns
df_language_check['Language_azure'].unique()
# in numbers
df_language_check['Language_azure'].value_counts()
# in detailed titles
df_language_check[df_language_check['Language_azure'] != 'English']
# make list of titles that are not English
not_english = list(df_language_check[df_language_check['Language_azure'] != 'English']["Title"])
# make list to pkl
import pickle
with open('not_english_title_azure.pkl', 'wb') as f:
    pickle.dump(not_english, f)

In [60]:
with open ('not_english_title_azure.pkl', 'rb') as fp:
    not_english2 = pickle.load(fp)
not_english2

['A Deanadh Im',
 'A Mhisg A Chur An Lolig Oirn',
 'Ag Argoint I Scoil Eigse',
 'Ag Críost An Síol',
 'Aisling Gheal',
 'Sleamhnán Bhaile An Mhuilinn',
 'Amhrán Ar Earraigh',
 'An Asal Cam',
 'An Bóithrín Buí',
 'An Buachaill Dreoite',
 'An Buachaillin Bui',
 'An Buachaillin Dreoite',
 'An Buinnean Bui',
 'An Chearc Ar Fad Is An Anairthe',
 'An Chúilfhionn March',
 'An Cnoc Bui',
 'An Comhra Donn',
 'An Draigheann',
 'An Dro',
 'An Droichead Bheag',
 'An Feochán',
 'An Leanbh Si',
 'An Mhaighdean Mhara',
 'An Peata Sa Chistineach',
 'An Phis Fhliuch',
 'An Páistín Fionn',
 'An T-Eas',
 'An tSeanbhean Bhocht',
 'An Tobar Glé',
 'An Tri Is A Rian',
 'Polska For Marit Fält',
 'Ar An Imeall',
 'Ardaigh Do Mheanna',
 'Bean An Tí Ar Lár',
 'Bean Dubh An Ghleanna',
 'An Bhean Cheile Nar Casadh Dhom',
 'Black Cat Of Cydweli',
 "Bouffard's",
 'Braigh Loch Iall',
 "Brefni O'Reilly",
 'Breith Chriosd',
 'Buachaill ón Éirne',
 'Bwced',
 'An Seanduine Dóite',
 'Caisleán An Óir',
 'Carraig An tSoip'

In [20]:
for idx, doc in enumerate(response):
    print(sample_200[idx], doc.primary_language.name)

The Devil In The Kitchen English
10th Bat Crossing Rhine English
1st August English
2D Or Not 2D English
The 30 Year English
30th Anniversary English
5:30am English
The 72nd Highlanders' Farewell To Aberdeen English
Tommy Peoples' English
79th's Farewell To Gibraltar English
93 Not Out English
A Barrel Of Knives English
A Birthday English
The Britches English
A Bottle Of Vodka, Twenty Marlboro Reds And £50 Cashback, Please English
A Breton In Paris English
Love At The Window English
Molly On The Shore English
A Country Wedding English
A Cup Of Tea English
Trip To The Quarry English
A Day And An Age English
The Newfoundland English
A Deanadh Im Irish
A Dram At The Wake English
The Fair Wind English
A Favourite Duet By Mr Holmes English
A Fig For A Kiss English
The Bag Of Spuds English
A Fish Out Of Water English
A Fisherman's Song For Attracting Seals English
A French Dance English
A Ghost Of His Former Self English
A Glint Of Silver English
A Good Night's Sleep English
A Great Big Sea 

In [23]:
response[-4], sample_200[-4]

(DetectLanguageResult(id=196, primary_language=DetectedLanguage(name=English, iso6391_name=en, confidence_score=1.0), warnings=[], statistics=None, is_error=False),
 'Allt-y-Caethiwed')